In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# pip show tensorflow

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, UpSampling2D, Flatten, Concatenate
from tensorflow.keras.layers import MaxPooling2D, Conv2D, BatchNormalization, Dropout
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler, ReduceLROnPlateau, ProgbarLogger
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, Callback

# Load CIFAR10 dataset

In [ ]:
cifar10 = tf.keras.datasets.cifar10

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170498071/170498071 [==============================] - 4s 0us/step


In [ ]:
x_train = x_train.astype('float32')
x_test= x_test.astype('float32')

In [ ]:
x_min = x_train.min()
x_max = x_train.max()

###Normalization to [0,1]

In [ ]:
x_train_normalized = (x_train - x_min)/(x_max - x_min) 
x_test_normalized = (x_test - x_min)/(x_max - x_min)

In [ ]:
print("Shape of x_train_normalized:", x_train_normalized.shape)
print("Shape of x_test_normalized:", x_test_normalized.shape)

Shape of x_train_normalized: (50000, 32, 32, 3)
Shape of x_test_normalized: (10000, 32, 32, 3)


###Normalization to normal distribution

In [ ]:
# Calculate per-channel mean and standard deviation on the training set
channel_means = x_train.mean(axis=(0,1,2))
channel_stds = x_train.std(axis=(0,1,2))

In [ ]:
x_train_normalized1 = (x_train - channel_means) / channel_stds
x_test_normalized1 = (x_test - channel_means) / channel_stds

In [ ]:
print("Shape of x_train_normalized:", x_train_normalized1.shape)
print("Shape of x_test_normalized:", x_test_normalized1.shape)

Shape of x_train_normalized: (50000, 32, 32, 3)
Shape of x_test_normalized: (10000, 32, 32, 3)


In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:
y_train_one_hot = to_categorical(y_train, num_classes=10)
y_test_one_hot = to_categorical(y_test, num_classes=10)

In [ ]:
y_train_one_hot.shape, y_test_one_hot.shape

((50000, 10), (10000, 10))

# Multi-Layer Perception

In [ ]:
# MinMax Normalization Used
x_train_flattened = x_train_normalized.reshape(x_train_normalized.shape[0], -1)
x_test_flattened = x_test_normalized.reshape(x_test_normalized.shape[0], -1)

### First trial

Simple two-layers with 128 nodes and 10 nodes for input and output layers.

In [ ]:
#Defines architecture of MLP
model = Sequential([
    Dense(128, activation='relu', input_shape=(x_train_flattened.shape[1],)),
    Dense(10, activation='softmax')
])
#Training
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)

In [ ]:
model.fit(x_train_flattened, y_train_one_hot, epochs=10, batch_size=32, validation_data=(x_test_flattened, y_test_one_hot))
test_loss, test_acc = model.evaluate(x_test_flattened, y_test_one_hot, callbacks=[early_stopping])
print(f'Test accuracy: {test_acc * 100:.2f}%')

Epoch 1/10
1563/1563 [==============================] - 12s 7ms/step - loss: 1.8858 - accuracy: 0.3199 - val_loss: 1.7558 - val_accuracy: 0.3699
Epoch 2/10
1563/1563 [==============================] - 11s 7ms/step - loss: 1.7343 - accuracy: 0.3771 - val_loss: 1.7020 - val_accuracy: 0.3868
Epoch 3/10
1563/1563 [==============================] - 12s 7ms/step - loss: 1.6886 - accuracy: 0.3969 - val_loss: 1.6494 - val_accuracy: 0.4129
Epoch 4/10
1563/1563 [==============================] - 11s 7ms/step - loss: 1.6594 - accuracy: 0.4058 - val_loss: 1.6331 - val_accuracy: 0.4146
Epoch 5/10
1563/1563 [==============================] - 10s 7ms/step - loss: 1.6383 - accuracy: 0.4141 - val_loss: 1.6124 - val_accuracy: 0.4266
Epoch 6/10
1563/1563 [==============================] - 12s 8ms/step - loss: 1.6253 - accuracy: 0.4177 - val_loss: 1.6486 - val_accuracy: 0.4055
Epoch 7/10
1563/1563 [==============================] - 11s 7ms/step - loss: 1.6100 - accuracy: 0.4252 - val_loss: 1.6030 - val_ac

### Second trial

Increase number of nodes to 256 for input layer and experimented with Leaky ReLU activation function

In [ ]:
model1 = Sequential([
    Dense(256, activation='leaky_relu', input_shape=(x_train_flattened.shape[1],)),
    Dense(10, activation='softmax')
])

model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model1.fit(x_train_flattened, y_train_one_hot, epochs=10, batch_size=32, validation_data=(x_test_flattened, y_test_one_hot),
           callbacks=[early_stopping])

Epoch 1/10
1563/1563 [==============================] - 23s 15ms/step - loss: 1.9688 - accuracy: 0.3119 - val_loss: 1.8291 - val_accuracy: 0.3413
Epoch 2/10
1563/1563 [==============================] - 21s 14ms/step - loss: 1.7818 - accuracy: 0.3699 - val_loss: 1.7732 - val_accuracy: 0.3586
Epoch 3/10
1563/1563 [==============================] - 21s 13ms/step - loss: 1.6969 - accuracy: 0.3987 - val_loss: 1.8521 - val_accuracy: 0.3542
Epoch 4/10
1563/1563 [==============================] - 21s 14ms/step - loss: 1.6511 - accuracy: 0.4183 - val_loss: 1.6266 - val_accuracy: 0.4238
Epoch 5/10
1563/1563 [==============================] - 21s 13ms/step - loss: 1.6023 - accuracy: 0.4329 - val_loss: 1.6264 - val_accuracy: 0.4251
Epoch 6/10
1563/1563 [==============================] - 20s 13ms/step - loss: 1.5730 - accuracy: 0.4414 - val_loss: 1.5571 - val_accuracy: 0.4405
Epoch 7/10
1563/1563 [==============================] - 22s 14ms/step - loss: 1.5375 - accuracy: 0.4548 - val_loss: 1.5802 -

In [ ]:
test_loss1, test_acc1 = model1.evaluate(x_test_flattened, y_test_one_hot)
print(f'Test accuracy: {test_acc1 * 100:.2f}%')

313/313 [==============================] - 1s 3ms/step - loss: 1.4969 - accuracy: 0.4674
Test accuracy: 46.74%


### Third trial

Increase number of layers to three layers with 256, 128 & 10 nodes respectively. Experimented with Tanh activation function

In [ ]:
model2 = Sequential([
    Dense(256, activation='tanh', input_shape=(x_train_flattened.shape[1],)),
    Dense(128, activation='tanh'),
    Dense(10, activation='softmax')
])

model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model2.fit(x_train_flattened, y_train_one_hot, epochs=10, batch_size=32, validation_data=(x_test_flattened, y_test_one_hot),
           callbacks=[early_stopping])

Epoch 1/10
1563/1563 [==============================] - 22s 14ms/step - loss: 1.8991 - accuracy: 0.3192 - val_loss: 1.7148 - val_accuracy: 0.3914
Epoch 2/10
1563/1563 [==============================] - 22s 14ms/step - loss: 1.7140 - accuracy: 0.3872 - val_loss: 1.6771 - val_accuracy: 0.3895
Epoch 3/10
1563/1563 [==============================] - 22s 14ms/step - loss: 1.6338 - accuracy: 0.4154 - val_loss: 1.6470 - val_accuracy: 0.4294
Epoch 4/10
1563/1563 [==============================] - 20s 13ms/step - loss: 1.5805 - accuracy: 0.4381 - val_loss: 1.5945 - val_accuracy: 0.4285
Epoch 5/10
1563/1563 [==============================] - 22s 14ms/step - loss: 1.5454 - accuracy: 0.4499 - val_loss: 1.6567 - val_accuracy: 0.4203
Epoch 6/10
1563/1563 [==============================] - 22s 14ms/step - loss: 1.5005 - accuracy: 0.4659 - val_loss: 1.5441 - val_accuracy: 0.4498
Epoch 7/10
1563/1563 [==============================] - 21s 13ms/step - loss: 1.4779 - accuracy: 0.4732 - val_loss: 1.4775 -

In [ ]:
test_loss2, test_acc2 = model2.evaluate(x_test_flattened, y_test_one_hot)
print(f'Test accuracy: {test_acc2 * 100:.2f}%')

313/313 [==============================] - 1s 3ms/step - loss: 1.4792 - accuracy: 0.4781
Test accuracy: 47.81%


### Fourth trial

Increase number of nodes for each layer to 512, 256 & 10 respectively, while using Leaky ReLU activation function.

In [ ]:
model5 = Sequential([
    Dense(512, activation='leaky_relu', input_shape=(x_train_flattened.shape[1],)),
    Dense(256, activation='leaky_relu'),
    Dense(10, activation='softmax')
])
model5.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model5.fit(x_train_flattened, y_train_one_hot, epochs=10, batch_size=32, validation_data=(x_test_flattened, y_test_one_hot),
           callbacks=[early_stopping])
test_loss5, test_acc5 = model5.evaluate(x_test_flattened, y_test_one_hot)

Epoch 1/10
1563/1563 [==============================] - 38s 24ms/step - loss: 1.9197 - accuracy: 0.3147 - val_loss: 1.8351 - val_accuracy: 0.3522
Epoch 2/10
1563/1563 [==============================] - 37s 24ms/step - loss: 1.7244 - accuracy: 0.3830 - val_loss: 1.6048 - val_accuracy: 0.4289
Epoch 3/10
1563/1563 [==============================] - 35s 22ms/step - loss: 1.6361 - accuracy: 0.4158 - val_loss: 1.6169 - val_accuracy: 0.4338
Epoch 4/10
1563/1563 [==============================] - 34s 22ms/step - loss: 1.5874 - accuracy: 0.4348 - val_loss: 1.5557 - val_accuracy: 0.4502
Epoch 5/10
1563/1563 [==============================] - 39s 25ms/step - loss: 1.5403 - accuracy: 0.4503 - val_loss: 1.5296 - val_accuracy: 0.4532
Epoch 6/10
1563/1563 [==============================] - 38s 24ms/step - loss: 1.5117 - accuracy: 0.4625 - val_loss: 1.5199 - val_accuracy: 0.4602
Epoch 7/10
1563/1563 [==============================] - 36s 23ms/step - loss: 1.4731 - accuracy: 0.4739 - val_loss: 1.5283 -

In [ ]:
print(f'Test accuracy: {test_acc5 * 100:.2f}%')

Test accuracy: 47.13%


### Best Model - MLP

Increase number of layers to four with 512, 256, 128 and 10 nodes respectively followed by batch normalization after each dense layer. Switched back to ReLU activation function

In [ ]:
model9 = Sequential([
    Dense(512, activation='relu', input_shape=(x_train_flattened.shape[1],)),
    BatchNormalization(),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dense(10, activation='softmax')
])

In [ ]:
early_stopping_MLP = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

In [ ]:
model9.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model9.fit(x_train_flattened, y_train_one_hot, epochs=50, batch_size=32, validation_data=(x_test_flattened, y_test_one_hot), callbacks=[early_stopping_MLP])

Epoch 1/50
  60/1563 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.2172 - loss: 2.4323

W0000 00:00:1710754135.849245     148 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1563/1563 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3381 - loss: 1.8759

W0000 00:00:1710754141.051562     147 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1710754141.833241     148 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1563/1563 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.3382 - loss: 1.8759 - val_accuracy: 0.3159 - val_loss: 1.9527
Epoch 2/50
  41/1563 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.4112 - loss: 1.6736

W0000 00:00:1710754142.567383     146 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.4269 - loss: 1.6051 - val_accuracy: 0.4159 - val_loss: 1.6533
Epoch 3/50
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.4573 - loss: 1.5201 - val_accuracy: 0.4685 - val_loss: 1.4905
Epoch 4/50
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.4845 - loss: 1.4540 - val_accuracy: 0.4762 - val_loss: 1.4791
Epoch 5/50
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.4942 - loss: 1.4122 - val_accuracy: 0.4507 - val_loss: 1.5534
Epoch 6/50
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.5131 - loss: 1.3620 - val_accuracy: 0.4578 - val_loss: 1.5316
Epoch 7/50
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.5289 - loss: 1.3305 - val_accuracy: 0.4343 - val_loss: 1.6237
Epoch 8/50
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.5332 - loss: 1.3049 - val_accuracy: 0.4774 - val_loss: 1.4898
Epoch 9/50
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.5501 - loss: 1.2698 - val_accurac

In [ ]:
test_loss, test_acc = model9.evaluate(x_test_flattened, y_test_one_hot)
print(f'Test accuracy: {test_acc * 100:.2f}%')

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5358 - loss: 1.3260
Test accuracy: 53.16%


# Convolutional Neural Network

In [ ]:
model_CNN = Sequential([
  # Convolutional layers
  Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3), padding='same'),
  BatchNormalization(),
  Conv2D(32, (3, 3), activation='relu'),
  BatchNormalization(),
  MaxPooling2D((2, 2)),
  Dropout(0.25),

  # Convolutional layers
  Conv2D(64, (3, 3), activation='relu'),
  BatchNormalization(),
  Conv2D(64, (3, 3), activation='relu'),
  BatchNormalization(),
  MaxPooling2D((2, 2)),
  Dropout(0.25),

  # Flatten and fully connected layers
  Flatten(),
  Dense(128, activation='relu'),
  BatchNormalization(),
  Dropout(0.5),
  Dense(10, activation='softmax')
])

In [ ]:
early_stopping_CNN = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

In [ ]:
model_CNN.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_CNN.fit(x_train_normalized, y_train_one_hot, epochs=20, batch_size=32, validation_data=(x_test_normalized, y_test_one_hot),
             callbacks=[early_stopping_CNN])

Epoch 1/20
  30/1563 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.8003 - loss: 0.5569

W0000 00:00:1710754781.383602     146 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1563/1563 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8208 - loss: 0.5067

W0000 00:00:1710754792.373574     148 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1710754793.440244     148 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1563/1563 ━━━━━━━━━━━━━━━━━━━━ 22s 8ms/step - accuracy: 0.8208 - loss: 0.5067 - val_accuracy: 0.8054 - val_loss: 0.5656
Epoch 2/20
  21/1563 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.8353 - loss: 0.4638

W0000 00:00:1710754794.448872     149 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.8254 - loss: 0.4987 - val_accuracy: 0.7917 - val_loss: 0.6248
Epoch 3/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.8324 - loss: 0.4856 - val_accuracy: 0.8065 - val_loss: 0.5630
Epoch 4/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.8334 - loss: 0.4795 - val_accuracy: 0.8131 - val_loss: 0.5505
Epoch 5/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.8354 - loss: 0.4699 - val_accuracy: 0.8133 - val_loss: 0.5506
Epoch 6/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.8368 - loss: 0.4703 - val_accuracy: 0.8148 - val_loss: 0.5620
Epoch 7/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.8388 - loss: 0.4640 - val_accuracy: 0.8063 - val_loss: 0.5840
Epoch 8/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.8418 - loss: 0.4587 - val_accuracy: 0.8030 - val_loss: 0.6016
Epoch 9/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.8416 - loss: 0.4574 - val_accurac

In [ ]:
test_loss, test_acc = model_CNN.evaluate(x_test_normalized, y_test_one_hot)
print(f'Test accuracy: {test_acc*100:.2f}%')

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8294 - loss: 0.5304
Test accuracy: 82.74%


## Pretrained ResNet50 architecture with transfer learning

##### Convolutional layers were frozen to preserve the pre-training weights

In [ ]:
model = Sequential([
    UpSampling2D(size=(7, 7), input_shape=(32, 32, 3)),     # Upsample images to the minimum size required by ResNet50
    ResNet50V2(include_top=False, weights='imagenet', pooling='avg'), # Load ResNet50V2 with pretrained weights, without the top layer
    Flatten(),
    Dense(1024, activation='relu'),
    Dense(10, activation='softmax')
])

In [ ]:
for layer in model.layers[:-3]:
    layer.trainable = False

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train_normalized, y_train_one_hot, batch_size=64, epochs=10, validation_data=(x_test_normalized, y_test_one_hot),
          callbacks=[early_stopping1])

Epoch 1/10
  1/782 ━━━━━━━━━━━━━━━━━━━━ 1:58:53 9s/step - accuracy: 0.1406 - loss: 2.7640

W0000 00:00:1710749837.277927     148 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step - accuracy: 0.8039 - loss: 0.5946

W0000 00:00:1710749937.583057     148 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1710749941.542360     146 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


782/782 ━━━━━━━━━━━━━━━━━━━━ 135s 161ms/step - accuracy: 0.8039 - loss: 0.5944 - val_accuracy: 0.8480 - val_loss: 0.4319
Epoch 2/10
  1/782 ━━━━━━━━━━━━━━━━━━━━ 2:01 156ms/step - accuracy: 0.7969 - loss: 0.4365

W0000 00:00:1710749962.965122     147 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


782/782 ━━━━━━━━━━━━━━━━━━━━ 117s 149ms/step - accuracy: 0.8948 - loss: 0.3005 - val_accuracy: 0.8635 - val_loss: 0.4049
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 116s 148ms/step - accuracy: 0.9180 - loss: 0.2301 - val_accuracy: 0.8655 - val_loss: 0.4089
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 116s 148ms/step - accuracy: 0.9404 - loss: 0.1685 - val_accuracy: 0.8744 - val_loss: 0.4118
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 116s 148ms/step - accuracy: 0.9616 - loss: 0.1112 - val_accuracy: 0.8688 - val_loss: 0.4461


#####Unfrozen the ResNet50 layers and continue training with a lower learning rate

In [ ]:
for layer in model.layers[:-3]:
    layer.trainable = True

In [ ]:
early_stopping2 = EarlyStopping(monitor='val_loss', patience=3)
model_checkpoint = ModelCheckpoint('Pretrained_model.keras', monitor='val_accuracy', save_best_only=True, verbose=1)

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train_normalized, y_train_one_hot, batch_size=64, epochs=10, validation_data=(x_test_normalized, y_test_one_hot),
         callbacks=[early_stopping2, model_checkpoint])

Epoch 1/10


W0000 00:00:1710750473.346639     149 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 485ms/step - accuracy: 0.8876 - loss: 0.3585

W0000 00:00:1710750852.768613     148 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1710750856.824439     146 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update



Epoch 1: val_accuracy improved from -inf to 0.93490, saving model to Pretrained_model.keras


W0000 00:00:1710750877.929919     146 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


782/782 ━━━━━━━━━━━━━━━━━━━━ 450s 519ms/step - accuracy: 0.8876 - loss: 0.3584 - val_accuracy: 0.9349 - val_loss: 0.2084
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 466ms/step - accuracy: 0.9741 - loss: 0.0774
Epoch 2: val_accuracy did not improve from 0.93490
782/782 ━━━━━━━━━━━━━━━━━━━━ 384s 490ms/step - accuracy: 0.9740 - loss: 0.0774 - val_accuracy: 0.9266 - val_loss: 0.2377
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 465ms/step - accuracy: 0.9825 - loss: 0.0546
Epoch 3: val_accuracy did not improve from 0.93490
782/782 ━━━━━━━━━━━━━━━━━━━━ 383s 490ms/step - accuracy: 0.9825 - loss: 0.0546 - val_accuracy: 0.9235 - val_loss: 0.2580
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 465ms/step - accuracy: 0.9819 - loss: 0.0556
Epoch 4: val_accuracy did not improve from 0.93490
782/782 ━━━━━━━━━━━━━━━━━━━━ 383s 490ms/step - accuracy: 0.9819 - loss: 0.0556 - val_accuracy: 0.9270 - val_loss: 0.2624


In [ ]:
test_loss, test_acc = model.evaluate(x_test_normalized, y_test_one_hot)
print(f'Test accuracy: {test_acc * 100:.2f}%')

  3/313 ━━━━━━━━━━━━━━━━━━━━ 19s 64ms/step - accuracy: 0.9271 - loss: 0.1633

W0000 00:00:1710752049.225165     148 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


313/313 ━━━━━━━━━━━━━━━━━━━━ 31s 64ms/step - accuracy: 0.9300 - loss: 0.2543
Test accuracy: 92.70%


In [ ]:
# loaded_model = load_model('/kaggle/working/Pre_trained.keras')